In [3]:
import os,shutil,importlib
import ecProject as dev
importlib.reload(dev)

projectPath = os.getcwd()+'/temp/'
# mp = dev.myProject(projectPath)
# mp.projectConfig
if os.path.isdir(projectPath):
    print(projectPath)
    shutil.rmtree(projectPath)
myProject = dev.makeProject(projectPath)

c:\Users\User\CANFLUX\EC_Processing_Toolkit/temp/


In [4]:
importlib.reload(dev)

myProject = dev.importData(
    projectPath,
    siteID = 'SCL',
    inputPath = [[r'E:\temp',s] for s in ['20240724','20240914']],
    # inputPath=[r'E:\temp\\','20240914'],
    # inputPath=r'E:\GSC_Work\SCL_2024',
    # mode='map',
    # fileType=['TOB3']
    )

E:\temp\20240724\Flux_Data361.dat
E:\temp\20240724\Flux_Data362.dat
E:\temp\20240724\Flux_Data432.dat
E:\temp\20240724\Flux_Data433.dat
E:\temp\20240724\MetaData50.dat
[████████████████████████████████████████████████████████████] 4/4 

E:\temp\20240724\MetaData51.dat
[████████████████████████████████████████████████████████████] 1/1 

E:\temp\20240724\MetaData52.dat
E:\temp\20240724\MetaData53.dat
E:\temp\20240724\MetaData54.dat
E:\temp\20240724\MetaData55.dat
E:\temp\20240724\Met_Data77.dat
[████████████████████████████████████████████████████████████] 5/5 

E:\temp\20240724\Met_Data78.dat
E:\temp\20240724\Met_Data79.dat
Confirm dateutil parser works!
[████████████████████████████████████████████████████████████] 3/3 

Confirm dateutil parser works!
[████████████████████████████████████████████████████████████] 1/1 

E:\temp\20240724\Pussy\MetaData55.dat
[████████████████████████████████████████████████████████████] 1/1 

[████████████████████████████████████████████████████████████]

In [328]:
import pandas as pd
import numpy as np

file = r"E:\temp\20240914\20750527-SHSC.WSM.SGT.240720_240913readout.csv"

class parseHoboCSV():
    def __init__(self,log=False):
        self.log=log
        
    def parse(self,file,mode=1,saveTo=None):
        self.mode = mode
        self.f = open(file,'r',encoding='utf-8-sig')
        T = self.f.readline().rstrip('\n')
        if T.startswith('"Plot Title: '):
            self.isHobo = True
        else:
            self.isHobo = False
            self.mode = -1
        
        if self.mode >= 0:
            self.Metadata = {}
            self.Metadata['Type'] = 'HOBOcsv'
            self.Metadata['Program'] = T.replace('"','').split('Plot Title: ')[-1]
            self.Metadata['Table'] = self.Metadata['Program']
            self.parseHeader()
            # if self.mode >= 1:
            #     self.readData()
        self.f.close()

    def parseHeader(self):
        H = self.f.readline().replace('#','RecordNumber').lstrip('"').rstrip('"\n').split('","')
        print(H)
        H = [h.lstrip().rstrip(')').replace('(',',').split(',') for h in H]
        L = [len(h) for h in H]
        data = [h if l == max(L) else [sh for sh in h] + ['' for i in range(max(L)-l)] for h,l in zip(H,L)]
        # data = [[sh for sh in h] + ['' for i in range(max(L)-l)] for h,l in zip(H,L)]
        self.Header = pd.DataFrame(data = data).T
        self.Header.columns = self.Header.iloc[0].values+self.Header.iloc[-1].values
        self.Header.columns = self.Header.columns.str.replace(' ','_').str.replace(':','').str.rstrip('_')
        self.Header = self.Header[1:-1].copy()
        self.Header.index.name=''
        self.Header.index=['Unit','Logger','Sensor']
        self.statusCols = ['Host_Connected', 'Stopped', 'End_Of_File']
        self.readData()
        self.Metadata['Frequency'] = str(int(np.median(np.diff(self.Timestamp))))+' S'
        self.Metadata['Timezone'] = self.Header['Date_Time']['Unit'].lstrip()
        
    def readData(self):
        self.Data = pd.read_csv(self.f,header=None)
        self.Data.columns = self.Header.columns
        self.Data[self.statusCols] = self.Data[self.statusCols].ffill(limit=1)
        keep = pd.isna(self.Data[self.statusCols]).all(axis=1)
        self.Data = self.Data.loc[keep].copy()
        self.Timestamp = np.array([x.timestamp() for x in pd.to_datetime(self.Data['Date_Time'],format='%y/%m/%d %H:%M:%S')])
        


pH = parseHoboCSV()
pH.parse(file)
if pH.isHobo:
    print(pH.Metadata)

['RecordNumber', 'Date Time, GMT+00:00', 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 5)', 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 25)', 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 50)', 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 100)', 'Host Connected (LGR S/N: 20750527)', 'Stopped (LGR S/N: 20750527)', 'End Of File (LGR S/N: 20750527)']
{'Type': 'HOBOcsv', 'Program': '20750527-SHSC.WSM.SGT.240720', 'Table': '20750527-SHSC.WSM.SGT.240720', 'Frequency': '1800 S', 'Timezone': 'GMT+00:00'}


In [329]:
# pH.Data
# pH.Header = [h for h in pH.Header]
pH.Header.loc['dataType'] = pH.Data.dtypes
pH.Header

,RecordNumber,Date_Time,Temp_LBL_5,Temp_LBL_25,Temp_LBL_50,Temp_LBL_100,Host_Connected,Stopped,End_Of_File
Unit,,GMT+00:00,°C,°C,°C,°C,LGR S/N: 20750527,LGR S/N: 20750527,LGR S/N: 20750527
Logger,,,LGR S/N: 20750527,LGR S/N: 20750527,LGR S/N: 20750527,LGR S/N: 20750527,,,
Sensor,,,SEN S/N: 20750527,SEN S/N: 20750527,SEN S/N: 20750527,SEN S/N: 20750527,,,
dataType,int64,object,float64,float64,float64,float64,object,object,object


array([ True,  True,  True,  True,  True,  True, False, False, False])

In [238]:
myProject.fileInventory

{'incoming': [{'Name': 'E:\\temp',
   'Child': [{'Name': '20240724',
     'Child': [{'Name': 'Flux_Data361', 'Ext': 'dat'},
      {'Name': 'Flux_Data362', 'Ext': 'dat'},
      {'Name': 'Flux_Data432', 'Ext': 'dat'},
      {'Name': 'Flux_Data433', 'Ext': 'dat'},
      {'Name': 'MetaData50', 'Ext': 'dat'},
      {'Name': 'MetaData51', 'Ext': 'dat'},
      {'Name': 'MetaData52', 'Ext': 'dat'},
      {'Name': 'MetaData53', 'Ext': 'dat'},
      {'Name': 'MetaData54', 'Ext': 'dat'},
      {'Name': 'MetaData55', 'Ext': 'dat'},
      {'Name': 'Met_Data77', 'Ext': 'dat'},
      {'Name': 'Met_Data78', 'Ext': 'dat'},
      {'Name': 'Met_Data79', 'Ext': 'dat'},
      {'Name': '20750527-SHSC.WSM.SGT.240720_240724readout', 'Ext': 'csv'},
      {'Name': '20750528-SHSC.SSM.SGT.240720_240724readout',
       'Ext': 'csv'}]}]}]}

In [189]:
f = myProject.fileInventory['incoming'][0]
# f['Name']
# f
def unpack(child,name,root=None):
    pth = []
    if type(child) == list:
        for c in child:
            if type(c) == dict and 'Child' in c.keys():
                pth,name,root = unpack(c['Child'],os.path.join(name,c['Name']),name)
            else:
                pth.append(os.path.join(name,c['Name']+'.'+c['Ext']))
        print(name)
        return(pth,name.split(os.path.join(root,''))[-1],root)
    else:
        if 'Child' in child.keys():
            pth,name,root = unpack(child['Child'],os.path.join(name,child['Name']),name)
        else:
            pth.append(os.path.join(name,child['Name']+'.'+child['Ext']))
        print(name)
        return(pth,name.split(os.path.join(root,''))[-1],root)
pth,name,root = unpack(f['Child'],f['Name'])
# pth,name,root = unpack(f['Child'][0],f['Name'])
print(pth)
print(name,root)

E:\temp\20240724
20240724
['E:\\temp\\20240724\\Flux_Data361.dat', 'E:\\temp\\20240724\\Flux_Data362.dat', 'E:\\temp\\20240724\\Flux_Data432.dat', 'E:\\temp\\20240724\\Flux_Data433.dat', 'E:\\temp\\20240724\\MetaData50.dat', 'E:\\temp\\20240724\\MetaData51.dat', 'E:\\temp\\20240724\\MetaData52.dat', 'E:\\temp\\20240724\\MetaData53.dat', 'E:\\temp\\20240724\\MetaData54.dat', 'E:\\temp\\20240724\\MetaData55.dat', 'E:\\temp\\20240724\\Met_Data77.dat', 'E:\\temp\\20240724\\Met_Data78.dat', 'E:\\temp\\20240724\\Met_Data79.dat', 'E:\\temp\\20240724\\20750527-SHSC.WSM.SGT.240720_240724readout.csv', 'E:\\temp\\20240724\\20750528-SHSC.SSM.SGT.240720_240724readout.csv']
20240724 E:\temp


In [68]:
import os,shutil,importlib
import development as dev
importlib.reload(dev)

projectPath = 'D:/temp/ArcticFlux'

for ID,tag,dIn in zip(['FIL','SCL'],
                  ['CR1000','Met_Data'],
                  ['D:\\MetStationData\\FishIsland',
                  'D:\\highfreq\SCL\\CardConvertDump'
                  ]
                  ):
    print(ID,tag)
    myProject = dev.dataDump(projectPath,
                            siteID=ID,
                            dIn=dIn,
                            fileType='TOA5',
                            searchTag = tag,
                            subPath='Met'
                            )


FIL CR1000
Searching  D:\MetStationData\FishIsland
Warning!!

Unable to parse date pattern from input file, proceeding without
Creating template:  D:\temp\ArcticFlux\rawSiteData/FIL/TOA5/Met/metaData.yml 

Edit this file before proceeding with import


Copying:[████████████████████████████████████████████████████████████] 1/1 

SCL Met_Data
Outstanding imports:  1
Searching  D:\highfreq\SCL\CardConvertDump
Creating template:  D:\temp\ArcticFlux\rawSiteData/SCL/TOA5/Met/metaData.yml 

Edit this file before proceeding with import


Copying:[████████████████████████████████████████████████████████████] 10/10 

Copying:[████████████████████████████████████████████████████████████] 31/31 

Copying:[████████████████████████████████████████████████████████████] 13/13 

